In [25]:
from IPython.display import display, HTML 
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

<b><font size="5" color="red">ch09. 파일 io(입출력) 프로그래밍 </font><b>
파일 : txt, pickle, csv, json, hdf5(h5)

In [6]:
try:
    f = open('data/ch09.txt', 'w')
except FileNotFoundError as e:
    print(e)
finally:
    f.close()

In [17]:
f = open('data/ch09.txt', 'w')
print('쓰기 가능한지 여부 :', f.writable())

쓰기 가능한지 여부 : True


In [18]:
f.write('Hello\nWorld')
f.close() # close() 쓰기한 내용이 저장에 필수!!!

In [21]:
# mode : 'r' 또는 'rt' : 파일이 없는 경우 예외, 파일이 있는 경우 text읽기 모드
#        'rb' : 바이너리 읽기 모두
#        'w' 또는 'wt' : text 쓰기 모드 (파일이 있으면 덮어씀, 파일이 없으면 파일 생성)
#        'wb' : 바이너리 쓰기 모드
#        'a' 또는 'at' : text 추가 모드 (파일이 있으면 append, 파일이 없으면 파일 생성)
#        읽기모드에서는 파일이 없으면 예외 / 쓰기모드에서는 폴더가 없으면 예외
# encoding
#    eur-kr (한글완성형) '믜' 표현 안됨
#    cp949 (확장된 한글완성형) : open()함수 기본 encoding방식(win)
#    utf-8 (한글조합형) : open()함수 기본 encoding방식(mac, linux), 주피터노트북 기본
with open('data/ch09.txt', 'w') as f:
    print('쓰기 가능한지 여부 :', f.writable())
    f.write('Hello\nPython\n')

쓰기 가능한지 여부 : True


In [30]:
with open('data/ch09.txt', 'a', encoding='cp949') as f:
    # 1방법
    f.write('홍길동, 33, 아무동9\n')
    f.write('김길동, 33, 아무동9\n')
    # 2방법
    textlist = ['홍길동, 33, 아무동9\n', '김길동, 33, 아무동9\n']
    for line in textlist:
        f.write(line)
    # 3방법
    f.writelines(textlist)

In [32]:
# 한줄씩 일기
with open('data/ch09.txt', 'r') as f:
    line = f.readline()
    print(line)

홍길동, 33, 아무동9



In [33]:
# 모든 줄을 일기
with open('data/ch09.txt', 'r') as f:
    line = f.readline()
    while line != '':
        print(line, end='')
        line = f.readline()

홍길동, 33, 아무동9
김길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9


In [34]:
# 모두 일기
with open('data/ch09.txt', 'r') as f:
    lines = f.read()
    print(lines)

홍길동, 33, 아무동9
김길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9
홍길동, 33, 아무동9
김길동, 33, 아무동9



In [35]:
lines

'홍길동, 33, 아무동9\n김길동, 33, 아무동9\n홍길동, 33, 아무동9\n김길동, 33, 아무동9\n홍길동, 33, 아무동9\n김길동, 33, 아무동9\n'

# 2절.  피클을 이용한 객체 저장 및 불러오기
## 2.1 형식이 있는 txt데이터 불러오기

In [43]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age = age
        self. email = email
        self. address = address
    def __str__(self):
        return '{}, {}, {}, {}'.format(self.name,
                                      '성년' if self.age>18 else '미성년',
                                      self.email,
                                      self.address)
    def as_dict(self):
        return {
            'name':self.name,
            'age':self.age,
            'email':self.email,
            'address':self.address
            }
    def __eq__(self, other):
        return self.name == other.name and \
                self.age == other.age and \
                self.email == other.email and \
                self.address == other.address

In [44]:
user1 = Member('홍', 20, 'a@a.com', '신림동')
user2 = Member('홍', 20, 'a@a.com', '신림동')
print(user1==user2)
print(user1.__eq__(user2)) # 같은 내용이면 True
id(user1), id(user2), type(user1), type(user2)

True
True


(2318853225488, 2318853213056, __main__.Member, __main__.Member)

In [51]:
# 형식이 있는 txt 파일 내용을 member list(피클 저장), 딕션너리 list(데이터프레임)로 저장
user_list = [] # member list
user_dict = [] # 딕셔너리 list

with open('data/ch09_member.txt', 'r', encoding='utf-8') as txt_file:
    lines = txt_file.readlines()
# print(lines)
for line in lines:
#    print(line)
    data = line.split(',')
#    print(data)
    name = data[0]
    age = int(data[1].strip()) # strip() : 좌우 공백(space, \t, \n) 제거
    email = data[2].strip()
    address = data[3].strip()
    
    user = Member(name, age, email, address)
    user_list.append(user)
    user_dict.append(user.as_dict())
#    user_dict.append(user.__dict__) # 위와 동일 기능


In [50]:
for user in user_list:
    print(user)
user_dict

홍길동, 성년, kildong@hong.com, 서울시 관악구
홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
신길동, 성년, shinkil@hong.com, 서울시 동작구


[{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울시 관악구'},
 {'name': '홍길숙',
  'age': 25,
  'email': 'kilsuk1@hong.com',
  'address': '서울시 영등포구'},
 {'name': '신길동', 'age': 30, 'email': 'shinkil@hong.com', 'address': '서울시 동작구'}]

## 2.2 피클링
- 객체 리스트(user_list) -> 피클파일로 쓰기
- 피클 파일을 읽기 -> 객체 리스트(load_user_list)

In [52]:
# 피클링을 이용한 객체를 저장하기
import pickle
with open('data/ch09_member.data', 'wb') as f:
    pickle.dump(user_list, f)

In [55]:
# 피클링을 이용한 파일에서 객체 데이터로 읽기
with open('data/ch09_member.data', 'rb') as f:
    load_user_list = pickle.load(f)

In [54]:
user_list == load_user_list

True

In [60]:
for idx in range(len(user_list)):
    print(idx, user_list[idx])
    print(idx, load_user_list[idx])

0 홍길동, 성년, kildong@hong.com, 서울시 관악구
0 홍길동, 성년, kildong@hong.com, 서울시 관악구
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
1 홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
2 신길동, 성년, shinkil@hong.com, 서울시 동작구
2 신길동, 성년, shinkil@hong.com, 서울시 동작구


In [61]:
for user, load_user in zip(user_list, load_user_list):
    print(user)
    print(load_user)

홍길동, 성년, kildong@hong.com, 서울시 관악구
홍길동, 성년, kildong@hong.com, 서울시 관악구
홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
홍길숙, 성년, kilsuk1@hong.com, 서울시 영등포구
신길동, 성년, shinkil@hong.com, 서울시 동작구
신길동, 성년, shinkil@hong.com, 서울시 동작구


In [65]:
for idx, (user, load_user) in enumerate(zip(user_list, load_user_list)):
    print(idx, user)
    print(idx, load_user)
    print(user==load_user)

True
True
True


In [67]:
result = []
for user, load_user in zip(user_list, load_user_list):
    result.append(user==load_user)
all(result)

True

# 3절. CSV형식 파일 읽기/쓰기
- csv파일 <--> 리스트데이터   3.1 / 3.2
- csv파일 <--> 딕셔너리데이터 3.3 / 3.4

## 3.1 reader

In [69]:
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8')as f:
    reader = csv.reader(f)
    result = list(reader)
print(result)

[['홍길동', '20', 'kildong@hong.com', '서울시 관악구'], ['김길동', '40', 'kimdong@hong.com', '서울시 영등포구'], ['신길동', '30', 'sindong@hong.com', '서울시 동작구']]


In [70]:
# ""(따옴표)가 없는 데이터는 numeric으로
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8')as f:
    reader = csv.reader(f,
                       quoting=csv.QUOTE_NONNUMERIC)
    result = list(reader)
print(result)

[['홍길동', 20.0, 'kildong@hong.com', '서울시 관악구'], ['김길동', 40.0, 'kimdong@hong.com', '서울시 영등포구'], ['신길동', 30.0, 'sindong@hong.com', '서울시 동작구']]


In [72]:
import csv
with open('data/ch09_member1.csv', 'r', encoding='utf-8')as f:
    reader = csv.reader(f,
                       quoting=csv.QUOTE_NONNUMERIC)
    result = list(reader)
dict_list = []
for data in result:
    dict_list.append({
        'name':data[0],
        'age':int(data[1]),
        'email':data[2],
        'address':data[3]
    })
dict_list

[{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울시 관악구'},
 {'name': '김길동',
  'age': 40,
  'email': 'kimdong@hong.com',
  'address': '서울시 영등포구'},
 {'name': '신길동', 'age': 30, 'email': 'sindong@hong.com', 'address': '서울시 동작구'}]

## 3.2 writer

In [74]:
user_list = [['홍길동', 20.0, 'kildong@hong.com', '서울시 관악구'],
             ['김길동', 40.0, 'kimdong@hong.com', '서울시 영등포구'],
             ['신길동', 30.0, 'sindong@hong.com', '서울시 동작구']]

In [75]:
with open('data/ch09_member1_write.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(user_list)

In [76]:
import csv
with open('data/ch09_member1_write.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC)
    writer.writerows(user_list)

## 3.3 DictReader

In [95]:
import csv
with open('data/ch09_member3.csv', 'r', encoding='utf-8') as f:
    dict_reader = csv.DictReader(f)
    dict_list = list(dict_reader)
    print(dict_list)

for row in dict_list:
    if row['job'] is None:
        print(row['name'], row['age'], row['email'], row['address'])
    else:
        print(row['name'], row['age'], row['email'], row['address'], row['job'])

[{'name': '홍길동', 'age': '20', 'email': 'h@h.com', 'address': '서울시 관악구', 'job': None}, {'name': '신길동', 'age': '40', 'email': 's@h.com', 'address': '서울시 영등포구', 'job': '팀장'}, {'name': '김길동', 'age': '30', 'email': 'k@h.com', 'address': '서울시 동작구', 'job': None}]
홍길동 20 h@h.com 서울시 관악구
신길동 40 s@h.com 서울시 영등포구 팀장
김길동 30 k@h.com 서울시 동작구


In [96]:
# header가 없는 csv파일(ch09_member1-cp.csv)을 딕셔너리로 읽기
with open('data/ch09_member1-cp.csv', encoding='cp949') as f:
    dict_reader = csv.DictReader(f, 
                                 fieldnames=['Name', 'Age', 'Email', 'Address', 'Job'], 
                                 restkey='Job')
    result = list(dict_reader)
    print(result)
    
for row in result:
    print(row)

[{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울시 관악구', 'Job': ''}, {'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '서울시 영등포구', 'Job': '팀장'}, {'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ''}]
{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울시 관악구', 'Job': ''}
{'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '서울시 영등포구', 'Job': '팀장'}
{'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ''}


## 3.4 DictWriter
- 딕셔너리 리스트 -> CSV파일

In [97]:
user1 = {'name': '홍길동', 'age': 22, 'email': 'a@a.com', 'address': '신림동'}
user2 = {'name': '신길동', 'age': 30, 'email': 'b@a.com', 'address': '신림동'}
user3 = {'name': '김길동', 'age': 42, 'email': 'c@a.com', 'address': '신림동'}
user_list = [user1, user2, user3]
fieldnames = list(user1.keys())

In [99]:
list(user1.keys())

['name', 'age', 'email', 'address']

In [101]:
with open('data/ch09_member4.csv', 'w', encoding='utf-8', newline='') as f:
    dict_writer = csv.DictWriter(f,
                                fieldnames=fieldnames)
    dict_writer.writeheader() # header 쓰기
    dict_writer.writerows(user_list)

In [102]:
with open('data/ch09_member4.csv', 'a', encoding='utf-8', newline='') as f:
    dict_writer = csv.DictWriter(f,
                                fieldnames=fieldnames)
    # dict_writer.writeheader() # header 쓰기
    dict_writer.writerows(user_list)

## csv <-> 데이터프레임

In [103]:
import pandas as pd
member = pd.read_csv('data/ch09_member3.csv') # encoding='utf-8' : default라 불필요
member

,name,age,email,address,job
0,홍길동,20,h@h.com,서울시 관악구,NaN
1,신길동,40,s@h.com,서울시 영등포구,팀장
2,김길동,30,k@h.com,서울시 동작구,NaN


In [104]:
type(member)

pandas.core.frame.DataFrame

# 4절. JSON 데이터 저장하고 불러오기(dump, load)
- 딕셔너리리스트 <-> JSON 파일(기본)
- 객체리스트 <-> JSON 파일
## 4.1 dump(파일 출력)

In [105]:
data = [{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울', 'Job': ''},
        {'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '인천', 'Job': '팀장'},
        {'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '경기', 'Job': ''}]

In [107]:
# ensure_ascii 매개변수
    # True  : 비ASCII문은 유니코드 형태로 저장
    # False : 비 ASII문자 원래 형태로 저장
import json
with open('data/ch09_member.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(data, # 딕셔너리 리스트
              jsonfile,
              ensure_ascii=False,
              indent='\t')

In [109]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age = age
        self. email = email
        self. address = address
    def __str__(self):
        return '{}, {}, {}, {}'.format(self.name,
                                      self.age,
                                      self.email,
                                      self.address)
    def as_dict(self):
        return {
            'name':self.name,
            'age':self.age,
            'email':self.email,
            'address':self.address
            }
    def __eq__(self, other):
        if isinstance(other, Member):
            return self.__str__() == other.__str__()
#             return self.name == other.name and \
#                     self.age == other.age and \
#                     self.email == other.email and \
#                     self.address == other.address
        else:
            return False

In [112]:
user1 = Member('홍길동', 22, 'a@a.com', '신림동')
user2 = Member('홍길동', 22, 'a@a.com', '신림동')
print(user1.__eq__(user2))
print(user1=='홍길동')

True
False


In [113]:
member_list = [Member('홍길동', 22, 'a@a.com', '서울'),
              Member('신길동', 32, 'a@a.com', '서울'),
              Member('김길동', 42, 'a@a.com', '서울')]

In [116]:
with open('data/ch09_member1.json', 'w', encoding='utf-8') as jsonfile:
    json.dump(member_list, # 객체리스트
             jsonfile,
             ensure_ascii=False,
             indent='\t',
             default=Member.as_dict # 객체를 딕션너리로 return하는 인스턴스 함수
             )

## 4.2 load(파일입력)
- json파일 -> 딕셔너리리스트(기본)
                 ↓
- json파일 -> 객체리스트

In [123]:
def as_member(dic): 
    '매개변수로 딕셔너리를 받아 Member 객체를 return'
    return Member(dic.get('name'),dic['age'],dic.get('email'),dic.get('address'))

In [124]:
member = as_member({'name':'김길동', 'age':20, 'email':'z@a.com', 'address':'관악'})
print(member)

김길동, 20, z@a.com, 관악


In [126]:
with open('data/ch09_member.json', 'r', encoding='utf-8') as f:
    load_data = json.load(f)
load_data

[{'Name': '홍길동',
  'Age': '20',
  'Email': 'kildong@hong.com',
  'Address': '서울',
  'Job': ''},
 {'Name': '김길동',
  'Age': '40',
  'Email': 'kimdong@hong.com',
  'Address': '인천',
  'Job': '팀장'},
 {'Name': '신길동',
  'Age': '30',
  'Email': 'sindong@hong.com',
  'Address': '경기',
  'Job': ''}]

In [132]:
with open('data/ch09_member1.json', 'r', encoding='utf-8') as f:
    load_member_list = json.load(f, object_hook=as_member)
for load_member in load_member_list:
    print(load_member)

홍길동, 22, a@a.com, 서울
신길동, 32, a@a.com, 서울
김길동, 42, a@a.com, 서울


### JSON -> 데이터 프레임

In [128]:
import pandas as pd
pd.read_json('data/ch09_member1.json')
# pd.함수() : encoding='utf-8' 기본값
# open(파일, 모드)함수 : encoding='cp949' 기본값

,name,age,email,address
0,홍길동,22,a@a.com,서울
1,신길동,32,a@a.com,서울
2,김길동,42,a@a.com,서울


In [130]:
pd.read_json('data/ch09_member.json', encoding='utf-8')

,Name,Age,Email,Address,Job
0,홍길동,20,kildong@hong.com,서울,
1,김길동,40,kimdong@hong.com,인천,팀장
2,신길동,30,sindong@hong.com,경기,


## 5절. 연습문제 
### 실습형 : 기능별 함수를 작성한다

In [133]:
class Custmer:
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
        
    def as_dic(self): # 객체를 딕셔너리데이터로 반환 (csv 파일 저장시)
        return {
            'name':self.name,
            'phone':self.phone,
            'email':self.email,
            'age':self.age,
            'grade':self.grade
            'etc':self.etc
            }
    
    def to_list_style(self): #객체를 list return([홍길동, 010-8999-9999, e@e.com, 20, 3, 까칠해])
        return {[
            self.name,
            self.phone,
            self.email,
            self.age,
            self.grade
            self.etc]
            }
    
    def __str__(self): # ***홍길동 010-8999-9999 e@e.com 20 까칠해
        return '{} {} {} {} {} {}'.format(self.name,
                                        self.phone,
                                        self.email,
                                        self.age,
                                        self.grade
                                        self.etc)
    
    def to_customer(row): # txt파일 내용 한줄 (홍길동 , 010 8999 9999, e@e.com, 20, 3, 까칠해)을 Customer 객체로 반환

# 0.실행하면 data/ch09_customers.txt 파일의 내용을 load(customer_list)
#      data/ch09_customers.txt이 존재하지 않으면
#          빈 data/ch09_customers.txt 파일을 생성하고
#              데이터는 customer_list=[]
def load_customers():

# 1.입력
def fn1_insert_customer_info():
    
# 2.전체 출력
def fn2_print_customers(customer_list):
    
# 3.삭제 (동명이인이 있다고 할지 동명이인이 없다고 할지 미리 정해놓고 구현)
def fn3_delete_customer(customer_list):
    
# 4.이름찾기 (동명이인이 있다고 할지 없다고 할지 정한대로 구현)
def fn4_search_customer(customer_list):
    
# 5.내보내기 (CSV)
def fn5_save_customer_csv(customer_list):
    
# 9.종료 (종료하기 전 customer_list 를 txt 파일에 저장하고 종료)
def fn9_save_customer_txt(customer_list):    
    

def main():
    global customer_list
    customer_list = load_customers() # ch09_customers.txt의 내용을 load
    while True:
        print("1:입력 "," 전체출력 "," 삭제 "," 이름찾기 "," 내보내기 (CSV)", " 종료 ", sep=' | ', end='')
        fn = int(input('메뉴선택 :'))
        if fn == 1:
            customer = fn1_insert_customer_info() # 입력받은 내용으로 customer 객체를 반환
            customer_list.append(customer)
        elif fn == 2:
            fn2_print_customers(customer_list)
        elif fn == 3:
            fn3_delete_customer(customer_list)
        elif fn == 4:
            fn4_search_customer(customer_list)
        elif fn == 5:
            fn5_save_customer_csv(customer_list)
        elif fn == 9:
            fn9_save_customer_txt(customer_list)
        break

if __name__ == '__main__':
    main()

NameError: name 'load_customers' is not defined